# WSI - ćwiczenie 7
*Tomasz Żebrowski*

Celem zadania było stworzenie narzędzia do wnioskowania przy użyciu sieci Bayesa i zademonstrowanie jego mozliwości na przykładzie systemu alarmowego.

Omawiany system można opisać za pomocą pięciu binarnych zmiennych losowych. Zależności między nimi są zgodne z tymi przedstawionymi na wykładzie, a na potrzeby wykorzystania w programie reprezentuję je w poniższy sposób:

In [1]:
# Tabela prawdopodobieństw
probability_table = {
    "T": [ # Wystąpienie trzęsienia ziemi
        (None, 0.02)
        ],
    "W": [ # Włamanie
        (None, 0.01)
        ],
    "A": [ # Załączenie alarmu
        ([("W", True), ("T", True)], 0.95),
        ([("W", True), ("T", False)], 0.94),
        ([("W", False), ("T", True)], 0.29),
        ([("W", False), ("T", False)], 0.001)
        ],
    "J": [ # Telefon od Jana
        ([("A", True)], 0.90),
        ([("A", False)], 0.05)
    ],
    "M": [ # Telefon od Magdy
        ([("A", True)], 0.70),
        ([("A", False)], 0.01)
    ],
}

Algorytm MCMC z próbkowaniem Gibbsa zaimplementowałem w pliku `bayes.py` jako metodę klasy `BayesNet`. Poniższy kod tworzy obiekt tej klasy reprezentujący sieć o zadanych połączeniach.

In [2]:
from bayes import BayesNet

net = BayesNet(probability_table)

Weryfikuję czy algorytm działa poprawnie wyznaczając z jego pomocą znane z założeń zadania prawdopodobieństwa.

In [3]:
results = {}
for variable in probability_table:
    if probability_table[variable][0][0] is None:
        results[f"P({variable})"] = net.mcmc({}, variable, 100000)
    else:
        for variant in probability_table[variable]:
            evidence = {}
            for evidence_element in variant[0]:
                evidence[evidence_element[0]] = evidence_element[1]
            results[f"P({variable}|{evidence})"] = net.mcmc(evidence, variable, 100000)

In [4]:
correct_probability = [0.02, 0.01, 0.95, 0.94, 0.29, 0.001, 0.90, 0.05, 0.70, 0.01]
print(f"{'Liczone prawdopodobieństwo':28}{'Wynik programu':17}{'Prawidłowy wynik':16}")
for index, result in enumerate(results):
    print(f"{result:32}{results[result]:4.4f}{correct_probability[index]:16.4f}")

Liczone prawdopodobieństwo  Wynik programu   Prawidłowy wynik
P(T)                            0.0217          0.0200
P(W)                            0.0141          0.0100
P(A|{'W': True, 'T': True})     0.9541          0.9500
P(A|{'W': True, 'T': False})    0.9391          0.9400
P(A|{'W': False, 'T': True})    0.2929          0.2900
P(A|{'W': False, 'T': False})   0.0005          0.0010
P(J|{'A': True})                0.8999          0.9000
P(J|{'A': False})               0.0470          0.0500
P(M|{'A': True})                0.7115          0.7000
P(M|{'A': False})               0.0097          0.0100


Widać, że dla tak dużej liczby iteracji (100000) wyniki wyznaczone przez program są bardzo bliskie wartościom rzeczywistym. Ponadto, wyznaczenie ich nie zajęło bardzo dużo czasu.

### Prezentacja możliwości programu
Oczywiście stworzone narzędzie pozwala na realizację wielu innych rodzajów zadań poza wyznaczeniem już znanych prawdopodobieństw.

Pierwszym z nich może być wyznaczenie łącznych prawdopodobieństw dla wszystkich zdarzeń (Przy braku wiedzy o systemie)

In [5]:
results = {}
for variable in probability_table:
    results[f"P({variable})"] = net.mcmc({}, variable, 100000)
print("Liczone prawdopodobieństwo\tWynik programu")
for result in results:
    print(f"{result:24}\t{results[result]:4.4f}")

Liczone prawdopodobieństwo	Wynik programu
P(T)                    	0.0196
P(W)                    	0.0130
P(A)                    	0.0154
P(J)                    	0.0647
P(M)                    	0.0200


W przypadku zdarzeń `T` i `W` wyniki są te same co poprzednio, również zgodne z założeniami zadania. Pozostałe prawdopodobieństwa zgadzają się z tymi, które można uzyskać licząć te prawdopodobieństwa ręcznie.

Program można zastosować do typów zadań wymienionych w wykładzie:
1. Zadanie diagnostyczne:   Magda zadzwoniła; Czy było włamanie?
2. Zadanie predykcyjne:     Było włamanie; Czy zadzwoni Jan?
3. Zadanie wyjaśniające:    Włączył się alarm i było trzęsienie ziemi; Czy nie było również włamania?
4. Zadanie mieszane:        Było trzęsienie ziemi i zadzwonił Jan; Czy włączył się też alarm?

In [6]:
# Zadanie 1
print("Zadanie diagnostyczne")
result_1 = net.mcmc({"M": True}, "W", 100000)
print(f"P(W|{{'M': True}}) = {result_1:4.4f}")

Zadanie diagnostyczne
P(W|{'M': True}) = 0.3036


In [7]:
# Zadanie 2
print("Zadanie predykcyjne")
jan = net.mcmc({"W": True}, "J", 100000)
print(f"P(J|{{'W': True}}) = {jan:4.4f}")

Zadanie predykcyjne
P(J|{'W': True}) = 0.8478


In [8]:
# Zadanie 3
print("Zadanie wyjaśniające")
result_3 = net.mcmc({"A": True, "T" : True}, "W", 100000)
print(f"P(W|{{'A': True, 'T': True}}) = {result_3:4.4f}")

Zadanie wyjaśniające
P(W|{'A': True, 'T': True}) = 0.0330


In [9]:
print(net.mcmc({"A": True}, "T", 100000))

0.3701625
